## Import the libraries we need

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import time
from splinter import Browser
from pprint import pprint
from datetime import datetime
import pymongo
from bson.json_util import loads
import json 
import pytz

# Setting upt our Mongo DB

In [3]:
print("Initialize PyMongo to work with MongoDBs")
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.travel_informers
collection = db.city_pics
print('Mongo Initialized')

Initialize PyMongo to work with MongoDBs
Mongo Initialized


# Initialize the Chromedriver

In [37]:
executable_path = {'executable_path':'D:/Chromedriver/chromedriver.exe'}
browser = Browser('chrome', **executable_path)

# get our list of cities
** And format the city names so that they match the url formats of wikipedia **  
 for example: https://en.wikipedia.org/wiki/San_Francisco  
 note the underscore

In [38]:
import pandas as pd
df = pd.read_csv('historical-hourly-weather-data/city_attributes.csv')

city_list = list(df.City[:30])
#print(city_list)

city_list = ['Vancouver', 'Portland', 'San Francisco', 'Seattle', 'Los Angeles', 'San Diego', 'Las Vegas', 
             'Phoenix', 'Albuquerque', 'Denver', 'San Antonio', 'Dallas', 'Houston', 'Kansas City', 'Minneapolis',
             'St._Louis', 'Chicago', 'Nashville', 'Indianapolis', 'Atlanta', 'Detroit', 'Jacksonville', 'Charlotte',
             'Miami', 'Pittsburgh', 'Toronto', 'Philadelphia', 'New York', 'Montreal', 'Boston']

# add underscors to city names that need them
clean_city_list = []
for city in city_list:
    if ' ' in city:
        clean_city_list.append(city.replace(' ', '_'))
    else:
        clean_city_list.append(city)

print("city names prepared for wikipedia.org: \n",clean_city_list)



city names prepared for wikipedia.org: 
 ['Vancouver', 'Portland', 'San_Francisco', 'Seattle', 'Los_Angeles', 'San_Diego', 'Las_Vegas', 'Phoenix', 'Albuquerque', 'Denver', 'San_Antonio', 'Dallas', 'Houston', 'Kansas_City', 'Minneapolis', 'St._Louis', 'Chicago', 'Nashville', 'Indianapolis', 'Atlanta', 'Detroit', 'Jacksonville', 'Charlotte', 'Miami', 'Pittsburgh', 'Toronto', 'Philadelphia', 'New_York', 'Montreal', 'Boston']


In [ ]:
base_url = "https://en.wikipedia.org/wiki/"

executable_path = {'executable_path':'D:/Chromedriver/chromedriver.exe'}
browser = Browser('chrome', **executable_path)

# clear out collection before scraping
# collection.delete_many({})

for city in clean_city_list:
    if collection.find_one({'city': city}):
        print(collection.find_one({'city':city}))
    else:
        record = {}
        record['city'] = city
        city_url = base_url + city
        browser.visit(city_url)
        soup = bs(browser.html, 'html.parser')

        #for title in soup.find_all("table", class_='infobox geography vcard'):
             #print(title.find_all('tr'))

        pics = []
        for pic in soup.find_all('a', class_='image'):
            img_url = 'https://en.wikipedia.org' + pic['href']
            print(img_url)
            browser.visit(img_url)
            pic_soup = bs(browser.html, 'html.parser')
            for element in pic_soup.find_all('div', class_='fullImageLink'):
                img_src = element.find('a').find('img')['src']
                pics.append(img_src)


        record['images'] = pics

        print(record)
        collection.insert_one(record)


https://en.wikipedia.org/wiki/File:Downtown_Vancouver_2017.jpg
https://en.wikipedia.org/wiki/File:Flag_of_Vancouver_(Canada).svg
https://en.wikipedia.org/wiki/File:Vancouverlogo.svg
https://en.wikipedia.org/wiki/File:GVRDVancouver.svg
https://en.wikipedia.org/wiki/File:Canada_British_Columbia_relief_location_map.jpg
https://en.wikipedia.org/wiki/File:Relief_map_of_Canada.png
https://en.wikipedia.org/wiki/File:North_America_laea_relief_location_map.jpg
https://en.wikipedia.org/wiki/File:Maple_Tree_Corner_Vancouver_1886.jpg
https://en.wikipedia.org/wiki/File:First_Vancouver_Council_Meeting_after_fire.jpg
https://en.wikipedia.org/wiki/File:RCMP_1938_sitdowner_strike.jpg
https://en.wikipedia.org/wiki/File:Stadtgliederung_Vancouver_2008.png
https://en.wikipedia.org/wiki/File:Stanley_Park_1999_Rain.jpg
https://en.wikipedia.org/wiki/File:Vancouver_aerial_view.jpg
https://en.wikipedia.org/wiki/File:Vancouver_dusk_pano.jpg
https://en.wikipedia.org/wiki/File:Robson_Square_Vancouver_02.JPG
https:

## Rendering the scraped images are randomized collages for each city
Now that we have our images stored in Mongo lets create a function that pulls randomly draws pics for a collage

In [ ]:
import random

def retrieve_pics(city):
    doc = collection.find_one({"city": city})
    num_images = len(doc['images'])
    randomized_selection = random.sample(6,)